In [576]:
# Fetch all the analysis objects
import psycopg2
import time
start = time.time()

In [577]:
try:
    conn = psycopg2.connect("dbname='sales' user='postgres' host='db.talentify.in' password='cx6ac54nmgGtLD1y'")
    print("Connected to database in {:.03f} seconds".format((time.time()- start)/1000))
except:
    print("I am unable to connect to the database")

Connected to database in 0.001 seconds


In [578]:
sql = "select analytics from task where status = 'COMPLETED' and length(analytics)>1000 order by id desc"
cur = conn.cursor()
print('Executing {}'.format(sql))
cur.execute(sql)
rows = cur.fetchall()

print("Fetched {} results in {:.03f} seconds".format(len(rows), (time.time()- start)/1000))

Executing select analytics from task where status = 'COMPLETED' and length(analytics)>5000 order by id desc
Fetched 48607 results in 0.113 seconds


In [579]:
class Snippet:
    def __init__(self, transcription, signal):
        self.transcription = transcription
        self.signal = signal
        
    def print(self):
        print("{} --> {}".format(self.transcription ,self.signal))
        
    def print_clean(self):
        print("{} --> {}".format(self.clean_text ,self.signal))
        
    def set_clean_text(self, clean_text):
        self.clean_text = clean_text
    
    def set_idx_arr(self, idx_arr):
        self.idx_arr = idx_arr
    
    def set_word_arr(self, word_arr):
        self.word_arr = word_arr
        
snips = [] 
import json
for row in rows:
    ser_ana = row[0]
    ana = json.loads(ser_ana)
    convs = ana['conversation']
    for conv in convs:
        if 'text' in conv:
            if len(conv['text'])>10:
                is_greeting = False
                texts.append([conv['text']])
                for signal in conv['signals']:
                    #print('\t'+signal['name'])
                    if 'greet' in signal['name'].lower():
                        is_greeting = True
                        break
                snips.append(Snippet(conv['text'], is_greeting))

In [ ]:
# Split snips into greeting/ non-greeting -> snip_greet (~5000), snip_no_greet (500000)
# Create new list with all snip_greet and equal number from snip_no_great -> strips_new (~10000)

In [580]:
import re
for snip in snips:
    #snip.set_clean_text(re.sub("[_,.':@#?!&$\"-]", '', snip.transcription).strip().lower())
    snip.set_clean_text(snip.transcription.strip().lower())

In [581]:
word_list = []
for snip in snips:
    for word in snip.clean_text.split(' '):
        if word not in word_list:
            word_list.append(word)

In [451]:
def word_array_to_word_index_array(word_array, word_list):
    word_index_array = []
    for word in word_array:
        word_index_array.append(word_list.index(word))
    return word_index_array

In [452]:
def sentence_to_word_array(sentence):
    return sentence.split(" ") 

In [453]:
for snip in snips:
    word_arr = sentence_to_word_array(snip.clean_text) #['hi', 'my', 'name', 'is', 'chirag']
    idx_arr = word_array_to_word_index_array(word_arr, word_list) # [143, 12, 1, 768, 2121]
    snip.set_idx_arr(idx_arr)
    snip.set_word_arr(word_arr)

In [454]:
import numpy as np
data = []
labels = []
for snip in snips:
    data.append(snip.idx_arr)
    if snip.signal==True:
        labels.append(1)
    else:
        labels.append(0)

In [455]:
test_data = []
test_labels = []
train_data = []
train_labels = []
# Split
import random
for snip in snips:
    ran = random.random() * 100
    if ran < 50:
        train_data.append(snip.idx_arr)
        if snip.signal==True:
            train_labels.append(1)
        else:
            train_labels.append(0)
    else:
        test_data.append(snip.idx_arr)
        if snip.signal==True:
            test_labels.append(1)
        else:
            test_labels.append(0)

In [456]:
data  = np.array(data)
labels  = np.array(labels)
test_data  = np.array(test_data)
test_labels  = np.array(test_labels)
train_data  = np.array(train_data)
train_labels  = np.array(train_labels)

In [457]:
def word_list_to_word_index(word_list):
    word_index = {}
    for word, index in enumerate(word_list):
        word_index[index] = word
    return word_index

word_index= word_list_to_word_index(word_list)


In [458]:
# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3


In [459]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

def encode_review(text, word_list):
    word_arr = sentence_to_word_array(text) #['hi', 'my', 'name', 'is', 'chirag']
    idx_arr = word_array_to_word_index_array(word_arr, word_list)

In [460]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [461]:
len(train_data[0]), len(train_data[1])

(256, 256)

In [462]:
print(train_data[0])

[ 0  9 10  9 11 12 13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [464]:
vocab_size = len(word_index.keys())
vocab_size

91083

In [465]:
import tensorflow as tf
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 16)          1457328   
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 1,457,617
Trainable params: 1,457,617
Non-trainable params: 0
_________________________________________________________________


In [466]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [468]:
split = int(len(train_data)/2.4)
x_val = train_data[:split]
partial_x_train = train_data[split:]

y_val = train_labels[:split]
partial_y_train = train_labels[split:]


In [470]:
print('x_val.shape ->'+str(x_val.shape))
print('partial_x_train.shape ->'+str(partial_x_train.shape))
print('y_val.shape ->'+str(y_val.shape))
print('partial_y_train.shape ->'+str(partial_y_train.shape))

x_val.shape ->(88709, 256)
partial_x_train.shape ->(124193, 256)
y_val.shape ->(88709,)
partial_y_train.shape ->(124193,)


In [471]:
train_labels.shape

(212902,)

In [472]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 124193 samples, validate on 88709 samples
Epoch 1/40
124193/124193 [==============================] - 10s 79us/step - loss: 0.3344 - acc: 0.9376 - val_loss: 0.1419 - val_acc: 0.9711
Epoch 2/40
124193/124193 [==============================] - 10s 78us/step - loss: 0.2272 - acc: 0.9376 - val_loss: 0.1392 - val_acc: 0.9711
Epoch 3/40
124193/124193 [==============================] - 10s 77us/step - loss: 0.2220 - acc: 0.9377 - val_loss: 0.1387 - val_acc: 0.9711
Epoch 4/40
124193/124193 [==============================] - 10s 78us/step - loss: 0.2113 - acc: 0.9382 - val_loss: 0.1313 - val_acc: 0.9709
Epoch 5/40
124193/124193 [==============================] - 10s 78us/step - loss: 0.1955 - acc: 0.9388 - val_loss: 0.1208 - val_acc: 0.9705
Epoch 6/40
124193/124193 [==============================] - 10s 78us/step - loss: 0.1804 - acc: 0.9394 - val_loss: 0.1066 - val_acc: 0.9703
Epoch 7/40
124193/124193 [==============================] - 10s 78us/step - loss: 0.1677 - acc: 0.9399 - val_

In [473]:
results = model.evaluate(test_data, test_labels)
print(results)

212220/212220 [==============================] - 3s 12us/step
[0.1077612258531179, 0.9595655451889549]


In [561]:
def encode_review(texts, word_list):
    idx_arrs = []
    for text in texts:
        word_arr = sentence_to_word_array(text)
        idx_arr = np.zeros(256) # word_index["<PAD>"]
        for i, w in enumerate(word_arr):
            idx_arr[i] = word_index[w]  
        idx_arrs.append(idx_arr)
    return np.array(idx_arrs)

In [568]:
d = encode_review(['how are you doing today'], word_list)
prediction = model.predict(d)

In [569]:
prediction

array([[0.00014148]], dtype=float32)

In [570]:
d.shape

(1, 256)

In [573]:
partial_x_train

array([[12919, 14470,   185, ...,     0,     0,     0],
       [28542, 17920,   230, ...,     0,     0,     0],
       [  230,   359,    21, ...,     0,     0,     0],
       ...,
       [  227,    37,   337, ...,     0,     0,     0],
       [   32,   116,  1924, ...,     0,     0,     0],
       [17267,    12, 91078, ...,     0,     0,     0]], dtype=int32)

In [575]:
d

array([[ 78., 162.,  15., 251., 245.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0., 